# Near PD Test vs. R and Higham's 1988 Paper

In [5]:
import cProfile
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pstats
import scipy.sparse
import time
from numpy.linalg import norm
from pstats import SortKey

import eblp
import eblp.eblp as cov
import eblp.ccpd as ccpd

np.set_printoptions(linewidth=150, precision=5, suppress=False)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Higham Paper Examples

In [6]:
# for i in range(10):
#     for j in range(2, 50):
#         a = np.random.randn(j, j)
#         b = eblp.near_pd.near_pd(a, eig_tol=eig_tol)
#         assert eblp.linalg.is_spd(b, eig_tol=eig_tol)


In [7]:
a = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])

In [8]:
eblp.near_pd.near_pd(a, eig_tol=0)

array([[0.17678, 0.25   , 0.17678],
       [0.25   , 0.35355, 0.25   ],
       [0.17678, 0.25   , 0.17678]])

## Comparison with R Funntion

In [13]:
from rpy2.robjects.packages import importr, data
import numpy as np
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri as rpyn

datasets = importr('datasets')
ostatus = data(datasets).fetch('occupationalStatus')['occupationalStatus']
ostatus_np = np.array(ostatus)

In [34]:
ostatus

50,16,12,...,177,71,106


In [35]:
ostatus_np

array([[ 50,  19,  26,   8,   7,  11,   6,   2],
       [ 16,  40,  34,  18,  11,  20,   8,   3],
       [ 12,  35,  65,  66,  35,  88,  23,  21],
       [ 11,  20,  58, 110,  40, 183,  64,  32],
       [  2,   8,  12,  23,  25,  46,  28,  12],
       [ 12,  28, 102, 162,  90, 554, 230, 177],
       [  0,   6,  19,  40,  21, 158, 143,  71],
       [  0,   3,  14,  32,  15, 126,  91, 106]], dtype=int32)

In [38]:
robjects.r.assign("a_array", rpyn.py2rpy(a))
result = robjects.r("as.array(Matrix::nearPD(a_array, eig.tol=1e-6)$mat)")

In [39]:
print(result)

          [,1]      [,2]      [,3]
[1,] 0.1767767 0.2500000 0.1767767
[2,] 0.2500000 0.3535534 0.2500000
[3,] 0.1767767 0.2500000 0.1767767



In [41]:
np.array(robjects.conversion.rpy2py(result))

array([[0.17678, 0.25   , 0.17678],
       [0.25   , 0.35355, 0.25   ],
       [0.17678, 0.25   , 0.17678]])